In [125]:
import numpy as np
import pandas as pd # csv files
import random

iris=pd.read_csv('iris.csv')

In [126]:
 def gauss(x,mean,std):
    # x - jedna z wartości sample [a,b,c,d]
    exponent = np.exp(-(x-mean)**2/(2*std**2))
    return (1/(np.sqrt(2*np.pi*std**2))*exponent)

In [127]:
def naive_bayess_clasifier(training_set, sample_to_classify, iris_species_list):
    data = ProcessingData(training_set)
    stds = data.std()
    means = data.mean()
    probabilities = {}
    for name in iris_species_list:
        probabilities[name]=1
        for key in sample_to_classify.keys():
            probabilities[name]*=(gauss(sample_to_classify[key],means[name][key],stds[name][key]))
    guess = max(probabilities, key= lambda x: probabilities[x])
    return guess

In [128]:
class ProcessingData:
    def __init__(self, flowers_table):
        self.flowers_table = flowers_table

    def stiffle(self):
        x = self.flowers_table.copy()
        for i in range(len(x)-1,0,-1):
            tmp = random.randint(0,i)
            x.iloc[i], x.iloc[tmp] = x.iloc[tmp], x.iloc[i]
        return ProcessingData(x)
    
    def normalize(self):
        flower_data = self.flowers_table.loc[:, self.flowers_table.columns!='variety']
        iris_species = self.flowers_table.loc[:, self.flowers_table.columns=='variety']
        normalized_df=(flower_data-flower_data.min())/(flower_data.max()-flower_data.min())
        normalized_df['variety']=iris_species
        return ProcessingData(normalized_df)    
    
    def mean(self):    
        mean = pd.DataFrame()
        for i, name in enumerate(self.flowers_table.variety.unique()):
            mean[name] = self.flowers_table[self.flowers_table['variety'] == name].mean(axis=0)  
        return mean
    
    def std(self): # można zastąpić p.std()
        self.mean()
        std = self.mean().copy()
        for variety in self.flowers_table['variety'].unique():
            for prop in ['sepal.length','sepal.width','petal.length','petal.width']:
                std[variety][prop]=self.flowers_table[self.flowers_table['variety']==variety][prop].sub(self.mean()[variety][prop]).pow(2).mean()
        return std
    
    def split(self,a):
        x = self.flowers_table
        n = len(x)
        tabA=x[0:int(a*n)]
        tabB=x[int(a*n):n]
        return tabA, tabB

In [129]:
Iris = ProcessingData(iris)

irisTrain, irisVal = Iris.stiffle().normalize().split(0.7)

types = ['Setosa','Versicolor','Virginica']
accuracy=0;
for i in irisVal.iloc:
    sample = pd.DataFrame()
    #print(i['sepal.length'].item)
    sample = {
        'sepal.length':i['sepal.length'],
        'sepal.width':i['sepal.width'],
        'petal.length':i['petal.length'],
        'petal.width':i['petal.width']
    }
    
    answear = NaiveBayes.classify(irisTrain,sample,types)
    if(answear == i.variety):
        accuracy+=1
print('accuracy = ',(accuracy/len(irisVal)*100),'%')

<ipython-input-128-d883fe36d355>:22: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean[name] = self.flowers_table[self.flowers_table['variety'] == name].mean(axis=0)


accuracy =  97.77777777777777 %
